# Taget / Error exploration with umap / hdbscan


The general idea of this notebook is to explore where we get high MAE in terms of clusters of the main feature u_in. We use umap and hdbscan to proceed to dimensionnality reduction and clustering. This allow for 2D plot and aggregation of MAE by cluster to priorize wich clusters we have to deal with. Another version of this notebook, including some changes, exploring pressure and MAE is available here (https://www.kaggle.com/lucasmorin/pressure-mae-exploration-with-umap-hdbscan/edit/run/76608318).

# Base construction

In [ ]:
import numpy as np
import pandas as pd

import optuna

import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717

import matplotlib.pyplot as plt
import tensorflow as tf, gc
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

import pickle

DEBUG = False
TRAIN_MODEL = False

df_train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df_test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

if DEBUG:
    n = 1000
    df_train = df_train[:80*n]

def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df

train = add_features(df_train)
test = add_features(df_test)

targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

# Infer train data to compare

In [ ]:
EPOCH = 300
BATCH_SIZE = 1024
NUM_FOLDS = 10

gpu_strategy = tf.distribute.get_strategy()

with gpu_strategy.scope():
    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
    train_preds = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold>0:
            break
        K.clear_session()
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        checkpoint_filepath = f"folds{fold}.hdf5"
        model = keras.models.load_model('../input/finetune-of-tensorflow-bidirectional-lstm/'+checkpoint_filepath)
        train_preds.append(model.predict(train, batch_size=BATCH_SIZE, verbose=2).squeeze().reshape(-1, 1).squeeze())
        del model
        gc.collect()

In [ ]:
if DEBUG: n=1000
else: n=75450
    
df = pd.DataFrame({'target': targets.flatten(), 'preds': np.mean(train_preds,axis=0), 'id': [i+1 for i in range(n) for e in range(80)]})
preds_by_id = np.mean(train_preds,axis=0).reshape(-1, 80)
df['u_out_mask'] = (train[:,:,2] == -1).flatten()
df['error'] = np.abs(df['target']-df['preds'])
df_masked = df.mask(~df['u_out_mask'])
MAE_id = df_masked.groupby('id').agg(np.nanmean)['error']

In [ ]:
MAE_file = "MAE_id.pkl"

with open(MAE_file,'wb') as f:
    pickle.dump(MAE_id, f)
    
MAE_id.to_csv("MAE_id.csv", index=False)

# Basic Exploration

In [ ]:
MAE_id.describe()

In [ ]:
np.log(MAE_id).hist(bins=100)

In [ ]:
MAE_id_sorted = MAE_id.sort_values(ascending=False)

# worst predictions

In [ ]:
for i in MAE_id_sorted.index[:10]:
    ind = np.int(i)
    ind_labels = (MAE_id.index.values == i)
    
    print('cluster:'+str(i) + ' MAE: '+str(np.round(MAE_id_sorted[i],3)))

    plt.figure(figsize=(24,8))
    
    plt.subplot(1, 3, 1)
    plt.plot(train[ind_labels,:,1].transpose());
    plt.title('u_in')
    
    plt.subplot(1, 3, 2)
    plt.plot(targets[ind_labels].transpose(), label='target');
    plt.plot(preds_by_id[ind_labels].transpose(), label='preds');
    plt.legend()
    plt.title('pressure')
    
    plt.subplot(1, 3, 3)
    plt.plot((targets[ind_labels] - preds_by_id[ind_labels]).transpose());
    plt.title('error')
    
    plt.show()

LTSM seems to be able to predict weird noise, but has some constant error on some ids. However such errors seems rather rare. The instance with 15 MAE only account for 0.0002 global MAE. We must find if some clusters of output have more important global error (less individual MAE but high number of instances).

# Explaining worst predictions

Worst errors seems to come from a shift in values. 
From the discussion here: https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/278935 we see that most of the time a change in R and C only translate in some shift. So my main hypothese for explaining the worst predictions was some noise in R & C that translate into shiting the predictions. See the discussion here:https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/280356

One way to test this hypothese was to try and switch R & C values and see if we get better predictions.
With at least 3-4 values of the top 10 MAE explained by a shift in C, this appears something to be worth investigating more. Maybe we can try to predict better R and C from u_in values.

In [ ]:
df_trainw = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df_trainw['breath_id'] = [i+1 for i in range(75450) for e in range(80)]

for i in MAE_id_sorted.index[:10]:
    
    ind = np.int(i)
    ind_labels = (MAE_id.index.values == i)
    
    print(ind)
    
    df = df_trainw[df_trainw.breath_id == np.int(i)].reset_index(drop=True)

    print('original R: '+str(df.R.unique()[0]))
    print('original C: '+str(df.C.unique()[0]))
    
    
    basew_all = pd.DataFrame()

    for R in [20, 50,  5]:
        for C in [50, 20, 10]:
            basew_temp = df.copy()
            basew_temp['R']=R
            basew_temp['C']=C
            basew_temp['breath_id'] = basew_temp['breath_id']*100000 + C * 1000 + R
            basew_all = basew_all.append(basew_temp)
    
    trainw = add_features(basew_all)
    trainw.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
    trainw = RS.transform(trainw)
    trainw = trainw.reshape(-1, 80, trainw.shape[-1])
    
    gpu_strategy = tf.distribute.get_strategy()

    with gpu_strategy.scope():
        kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
        trainw_preds = []
        for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
            if fold>0:
                break
            K.clear_session()
            print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
            checkpoint_filepath = f"folds{fold}.hdf5"
            model = keras.models.load_model('../input/finetune-of-tensorflow-bidirectional-lstm/'+checkpoint_filepath)
            trainw_preds.append(model.predict(trainw, batch_size=BATCH_SIZE, verbose=2).squeeze().reshape(-1, 1).squeeze())
            del model
            gc.collect()
    
    preds = trainw_preds[0]
    
    plt.figure(figsize=(16,8))
    
    plot_max = np.max([np.max(preds), np.max(df.pressure), np.max(preds_by_id[ind_labels])])
    plot_min = np.min([np.max(preds), np.min(df.pressure), np.min(preds_by_id[ind_labels])])
    
    
    plt.subplot(1, 2, 1)
    plt.plot(df.pressure, label='target');
    plt.plot(preds_by_id[ind_labels].transpose(), label='preds');
    plt.ylim((np.min(plot_min), np.max(plot_max))) 
    plt.legend()
    plt.title('pressure')
    
    plt.subplot(1, 2, 2)
    plt.plot(preds.reshape(-1,80).transpose(), label = ('R '+basew_all.R.astype('str') +' - C '+ basew_all.C.astype('str')).unique());
    plt.ylim((np.min(plot_min), np.max(plot_max))) 
    plt.title('predicted_pressure')
    plt.legend()
    plt.show()
    
del trainw

# UMAP / Hdbscan

In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/hdbscan0827-whl/hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ hdbscan

# exploration on u_in - UMAP Embedding - hdbscan clustering

As the idea is to observe the main feature, we embed u_in with umap then we fit hdbscan on the embedding. Maybe better to fit hdbscan on the original data...

In [ ]:
%%time

import hdbscan
import umap

n=75450

# Switch to do the exploration on pressure
X = train[:,:,1]
#X = targets

reducer = umap.UMAP(random_state=42, n_components=2)
embedding = reducer.fit_transform(X)
clusterer = hdbscan.HDBSCAN(prediction_data=True, min_cluster_size = 50).fit(embedding)
u, counts = np.unique(clusterer.labels_, return_counts=True)

print(u)
print(counts)

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=clusterer.labels_, edgecolors='none', cmap='jet');

Doesn't seems to good of a clustering... there seems to be lot of 'outliers'.

# Plot error - log_scale

In [ ]:
import matplotlib.colors as colors
plt.figure(figsize=(10, 8))
plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=MAE_id[:n], edgecolors='none', cmap='jet', norm=colors.LogNorm(vmin=MAE_id[:n].quantile(0.05), vmax=MAE_id[:n].quantile(0.95)));
plt.colorbar();

# MAE by cluster

not exact as mean of MAE is not exactly the evaluation metric (due to u_out not starting at the same time), but should be ok for exploration.

In [ ]:
MAE_by_cluster = pd.DataFrame({'cluster':clusterer.labels_,'MAE':MAE_id})

MAE_cluster = MAE_by_cluster.groupby('cluster').agg(np.sum)['MAE']

#remove some base MAE, then divide by gloabl number
MAE_cluster_global = (MAE_cluster - counts * 0.1)/counts.sum()
MAE_cluster_global = MAE_cluster_global.sort_values(ascending=False)

In [ ]:
MAE_cluster_global

In [ ]:
for i in MAE_cluster_global.index[:10]:
    ind_labels = (clusterer.labels_ == i)
    print('cluster:'+str(i) + ' Global MAE: '+str(np.round(MAE_cluster_global[i],4)) + ' count: '+str(np.sum(ind_labels)))

    plt.figure(figsize=(24,6))
    
    plt.subplot(1, 4, 1)
    plt.scatter(embedding[:, 0], embedding[:, 1], s=5+100*ind_labels, c=ind_labels, edgecolors='none', cmap='viridis');
    plt.title('position')
    
    plt.subplot(1, 4, 2)
    plt.plot(train[ind_labels,:,1].transpose());
    plt.title('u_in')
    
    plt.subplot(1, 4, 3)
    plt.plot(targets[ind_labels].transpose());
    plt.title('pressure')
    
    plt.subplot(1, 4, 4)
    plt.plot((targets[ind_labels] - preds_by_id[ind_labels]).transpose());
    plt.title('error: target-preds')
    
    plt.show()

We have found some clusters with high MAE. The cluster 4 with 0.52 MAE and 360 instances account for 0.002 global MAE, while a single instance with 15 MAE account for 0,0002 of global MAE. It appears to be 10 times more important to deal with this cluster than to deal with the higher individual MAE. 

Regarding specific clusters and how to deal with them, some spike of errors seems concentrated. We might want to try some adapted features. 

# Study with u_out masking

We mask u_in with ~u_out to see if we get a better clustering.

In [ ]:
%%time

import hdbscan
import umap

n=75450

# Switch to do the exploration on pressure
X = train[:,:,1]
X_mask = X*((train[:,:,2]==-1)+0)
#X = targets

reducer_mask = umap.UMAP(random_state=42, n_components=2)
embedding_mask = reducer_mask.fit_transform(X_mask)
clusterer_mask = hdbscan.HDBSCAN(prediction_data=True, min_cluster_size = 50).fit(embedding_mask)
u_mask, counts_mask = np.unique(clusterer_mask.labels_, return_counts=True)

print(u_mask)
print(counts_mask)

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(embedding_mask[:, 0], embedding_mask[:, 1], s=5, c=clusterer_mask.labels_, edgecolors='none', cmap='jet');

In [ ]:
import matplotlib.colors as colors
plt.figure(figsize=(10, 8))
plt.scatter(embedding_mask[:, 0], embedding_mask[:, 1], s=5, c=MAE_id[:n], edgecolors='none', cmap='jet', norm=colors.LogNorm(vmin=MAE_id[:n].quantile(0.05), vmax=MAE_id[:n].quantile(0.95)));
plt.colorbar();

# Study by R and C

In [ ]:
df_rc = df_train[['breath_id','R','C']].groupby('breath_id').agg(np.min)

for r in [5,20,50]:
    for c in [10,20,50]:
        
        print('r: '+str(r)+', c: '+str(c))
        ind_rc = (df_rc.R==r) & (df_rc.C==c)

        X = targets[ind_rc]
        #u_in
        X_train = train[ind_rc,:,1]
        X_targets = targets[ind_rc]
        X_preds = preds_by_id[ind_rc]

        reducer = umap.UMAP(random_state=42, n_components=2)
        
        # switch to embed on u_in (not tested)
        embedding = reducer.fit_transform(X_train)
        #embedding = reducer.fit_transform(X)

        clusterer = hdbscan.HDBSCAN(prediction_data=True, min_cluster_size = 20).fit(embedding)
        u, counts = np.unique(clusterer.labels_, return_counts=True)

        plt.figure(figsize=(20, 8))
        plt.subplot(1, 2, 1)
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=clusterer.labels_, edgecolors='none', cmap='jet');
        plt.title('clusters')
        plt.subplot(1, 2, 2)
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=MAE_id[ind_rc.values], edgecolors='none', cmap='jet', norm=colors.LogNorm(vmin=1e-2, vmax=1e0));
        plt.title('MAE')
        plt.colorbar();
        plt.show();
        
        
        MAE_by_cluster = pd.DataFrame({'cluster':clusterer.labels_,'MAE':MAE_id[ind_rc.values]})
        MAE_cluster = MAE_by_cluster.groupby('cluster').agg(np.sum)['MAE']
        #remove some base MAE, then divide by gloabl number
        MAE_cluster_global = (MAE_cluster - counts * 0.15)/counts.sum()
        MAE_cluster_global = MAE_cluster_global.sort_values(ascending=False)
        
        
        for i in MAE_cluster_global.index[:3]:
            ind_labels = (clusterer.labels_ == i)
            print('cluster:'+str(i) + ' Global MAE: '+str(np.round(MAE_cluster_global[i],4)) + ' count: '+str(np.sum(ind_labels)))

            plt.figure(figsize=(24,6))
            plt.subplot(1, 4, 1)
            plt.scatter(embedding[:, 0], embedding[:, 1], s=5+100*ind_labels, c=ind_labels, edgecolors='none', cmap='viridis');
            plt.title('position')
            plt.subplot(1, 4, 2)
            plt.plot(X_train[ind_labels,:].transpose());
            plt.title('u_in')
            plt.subplot(1, 4, 3)
            plt.plot(X_targets[ind_labels].transpose());
            plt.title('pressure')
            plt.subplot(1, 4, 4)
            plt.plot((X_targets[ind_labels] - X_preds[ind_labels]).transpose());
            plt.title('error: target-preds')
            plt.show()
        
        
        
        

Some observations :

    - High Errors appear quite clustered for some R&C values. 
    
    - This does seems exploitable as we identify some cluster that seems to be responsible for more than 0.01 MAE gloably.
    
    - There might be some work to get better parameters for hdbscan (the three clusters for r: 5, c: 50 should be one). 
    
    - From the u_in graph and further inverstigation (coloring by first and last u_in - see below) the weirdest cluster appears linked to first and last values of u_in. 

# Role of u_in_first, u_in_last

In [ ]:
df_rc = df_train[['breath_id','R','C']].groupby('breath_id').agg(np.min)

for r in [20]:
    for c in [50]:
        
        print('r: '+str(r)+', c: '+str(c))
        ind_rc = (df_rc.R==r) & (df_rc.C==c)

        X = targets[ind_rc]
        #u_in
        X_train = train[ind_rc,:,1]
        X_targets = targets[ind_rc]
        X_preds = preds_by_id[ind_rc]

        reducer = umap.UMAP(random_state=42, n_components=2)
        embedding = reducer.fit_transform(X)

        clusterer = hdbscan.HDBSCAN(prediction_data=True, min_cluster_size = 20).fit(embedding)
        u, counts = np.unique(clusterer.labels_, return_counts=True)

        plt.figure(figsize=(20, 8))
        plt.subplot(1, 2, 1)
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=clusterer.labels_, edgecolors='none', cmap='jet');
        plt.title('clusters')
        plt.subplot(1, 2, 2)
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=MAE_id[ind_rc.values], edgecolors='none', cmap='jet', norm=colors.LogNorm(vmin=1e-2, vmax=1e0));
        plt.title('MAE')
        plt.colorbar();
        plt.show();
        
        plt.figure(figsize=(20, 8))
        plt.subplot(1, 2, 1)
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=X_train[:,1], edgecolors='none', cmap='jet');
        plt.title('u_in_first')
        plt.colorbar();
        plt.subplot(1, 2, 2)
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5, c=X_train[:,-1], edgecolors='none', cmap='jet');
        plt.title('u_in_last')
        plt.colorbar();
        plt.show();

 My idea was to  get the first and last values separately. I implemented some dedicated functions. But it doesn't exactly seems optimal to build constant time series for LSTM. Maybe we can build a second head that would take individual time series features. From my time series feature engineering notebook here :https://www.kaggle.com/lucasmorin/time-series-agregation-functions:

In [ ]:
get_first = lambda x: x.iloc[0]
get_first.__name__ = 'get_first'

get_last = lambda x: x.iloc[-1]
get_last.__name__ = 'get_last'

get_first_fn = [get_first,get_last]

create_feature_dict = {
    'u_in': get_first_fn,
}

train_features = df_train.groupby('breath_id').agg(create_feature_dict)
train_features.columns = ['_'.join(col) for col in train_features.columns]

train_features.head()